<a href="https://colab.research.google.com/github/ErikFantomex/Chatterbot-TIAI/blob/main/Chatbot_TAIA_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocesamiento

In [ ]:
!pip install chatterbot
!pip install chatterbot-corpus

In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')
import re
import requests
from bs4 import BeautifulSoup
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:

#iniciamos
url = 'https://ai.googleblog.com/2022/04/pathways-language-model-palm-scaling-to.html'
page = requests.get(url)
page = page.text
soup = BeautifulSoup(page, features="lxml")
for script in soup(["script", "style"]):
    script.decompose()
raw_text  = soup.get_text()

cleaned = re.sub(r"(?is)<(script|style).*?>.*?(\1>)", "", raw_text.strip())
cleaned = re.sub(r"(?s)[\n]?", "", cleaned)
cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
cleaned = re.sub(r" ", " ", cleaned)
cleaned = re.sub(r"  ", " ", cleaned)
cleaned = re.sub("\n", " ", cleaned)
cleaned = cleaned.strip()

tokens = nltk.sent_tokenize(cleaned)
for token in tokens:
  print('-> ', token, '\n')
     


In [ ]:
url = 'https://ai.googleblog.com/2022/04/pathways-language-model-palm-scaling-to.html'
page = requests.get(url)
page = page.text
soup = BeautifulSoup(page, features="lxml")
for script in soup(["script", "style"]):
    script.decompose()
raw_text  = soup.get_text()

cleaned = re.sub(r"(?is)<(script|style).*?>.*?(\1>)", "", raw_text.strip())
cleaned = re.sub(r"(?s)[\n]?", "", cleaned)
cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
cleaned = re.sub(r" ", " ", cleaned)
cleaned = re.sub(r"  ", " ", cleaned)
cleaned = re.sub("\n", " ", cleaned)
cleaned = re.sub("'", " ", cleaned)
cleaned = re.sub("\"", " ", cleaned)
cleaned = cleaned.strip()
     


# Vectorizacion

In [ ]:
sent_tokens = nltk.sent_tokenize(cleaned)
word_tokens = nltk.word_tokenize(cleaned)

In [ ]:
import math

total_ocurrences = {}
occured_in_n_docs = {}

for sentence in sent_tokens:
  for word in word_tokens:
 
    c = sentence.count(word)
 
    try:
      total_ocurrences[word] += c
    except KeyError:
      total_ocurrences[word] = c 
 
    if c > 0:
      try:
        occured_in_n_docs[word] += 1
      except KeyError:
        occured_in_n_docs[word] = 1
 

tfidf = []
i = 0
for sentence in sent_tokens:
  w = {"sentence_n": i}
  for word in word_tokens:
    # w = tf * log(N/df)
    tf = sentence.count(word)
    N = total_ocurrences[word]
    df = occured_in_n_docs[word]
    if tf == 0:
      w[word] = 0
    else:
      w[word] = tf * math.log(N/df)
  tfidf.append(w)
  i += 1


In [ ]:
pd.DataFrame(tfidf)
     

# Similaridad de coseno

In [ ]:
url = 'https://ai.googleblog.com/2022/04/pathways-language-model-palm-scaling-to.html'
page = requests.get(url)
page = page.text
soup = BeautifulSoup(page, features="lxml")
for script in soup(["script", "style"]):
    script.decompose()
raw_text  = soup.get_text()

cleaned = re.sub(r"(?is)<(script|style).*?>.*?(\1>)", "", raw_text.strip())
cleaned = re.sub(r"(?s)[\n]?", "", cleaned)
cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
cleaned = re.sub(r" ", " ", cleaned)
cleaned = re.sub(r"  ", " ", cleaned)
cleaned = re.sub("\n", " ", cleaned)
cleaned = re.sub("'", " ", cleaned)
cleaned = re.sub("\"", " ", cleaned)
cleaned = cleaned.strip()

sent_tokens = nltk.sent_tokenize(cleaned)

In [ ]:

similaridades = []
for sentece in sent_tokens:
  row = []
  A = {w for w in nltk.word_tokenize(sentece)}
  for another_sentence in sent_tokens:
    l1 =[]
    l2 =[]
    B = {w for w in nltk.word_tokenize(another_sentence)}
    v = A.union(B) 
    for w in v:
      l1.append(1 if w in A else 0)
      l2.append(1 if w in B else 0)
    c = 0
    for i in range(len(v)):
        c+= l1[i]*l2[i]
    row.append(c / float((sum(l1) * sum(l2)) ** 0.5))
  similaridades.append(row)

In [ ]:
pd.DataFrame(similaridades)

# Chatbot

In [ ]:
url_list = [
            "https://es.wikipedia.org/wiki/Ucr%C3%A2nia",
            "https://es.wikipedia.org/wiki/Rusia",
            "https://es.wikipedia.org/wiki/Invasi%C3%B3n_rusa_de_Ucrania_de_2022",
            "https://es.wikipedia.org/wiki/Guerra_em_Donbas",
            "https://es.wikipedia.org/wiki/Crimea",
            "https://cnnespanol.cnn.com/2022/12/08/ultima-hora-noticias-guerra-rusia-ucrania-orix-58/",
            "https://cnnespanol.cnn.com/2022/11/09/jerson-rusia-guerra-ucrania-orix/",
            "https://cnnespanol.cnn.com/2022/12/05/ultima-hora-noticias-guerra-rusia-ucrania-orix-55/",
            "https://cnnespanol.cnn.com/",
            "https://www.nytimes.com/es/",
            "https://stackoverflow.com/questions/50454716/cannot-properly-import-the-chatterbot-and-its-corpus"
            ]


all_senteces = []
for url in url_list:
  page = requests.get(url)
  page = page.text
  soup = BeautifulSoup(page, features="lxml")
  for script in soup(["script", "style"]):
      script.decompose()
  raw_text  = soup.get_text()

  cleaned = re.sub(r"(?is)<(script|style).*?>.*?(\1>)", "", raw_text.strip())
  cleaned = re.sub(r"(?s)[\n]?", "", cleaned)
  cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
  cleaned = re.sub(r" ", " ", cleaned)
  cleaned = re.sub(r"  ", " ", cleaned)
  cleaned = re.sub("\n", " ", cleaned)
  cleaned = re.sub("'", " ", cleaned)
  cleaned = re.sub("\"", " ", cleaned)
  cleaned = cleaned.strip()
  sent_tokens = nltk.sent_tokenize(cleaned)
  for token in sent_tokens:
    all_senteces.append(token)

In [ ]:
bot = ChatBot(
    'BotRepor',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    database_uri='sqlite:///database.sqlite3'
    )
    
chat = ListTrainer(bot)
chat.train(all_senteces)

List Trainer: [                    ] 0%

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


List Trainer: [####################] 100%


In [ ]:
while True:
  response = bot.get_response(input("Pregunta: "))
  print("[Confianza en la respuesta: {:}%]\nRespuesta:".format(100*float(response.confidence)), response)

Pregunta: rusia
[Confianza en la respuesta: 7.000000000000001%]
Respuesta: LRT.
Pregunta: >Hora dcdmx
[Confianza en la respuesta: 0.0%]
Respuesta: [368] Em 27 de fevereiro, mais de 100 mil pessoas se reuniram em Berlim para protestar contra a invasão da Rússia.
Pregunta: Hora cdmx
[Confianza en la respuesta: 0.0%]
Respuesta: [130]​Se han empleado varias tácticas de resistencia civil desarmada en toda Ucrania.


KeyboardInterrupt: ignored